In [1]:
import requests
import pandas as pd

import json

In [78]:
exportacoes_mundo_2023 = pd.read_excel('./dados/comex/Exportação  Mundo 2023 NCM.xlsx')
exportacoes_usa_2023 = pd.read_excel('./dados/comex/Exportação USA 2023 NCM.xlsx.xlsx')


In [79]:
exportacoes_mundo_2023['cod_ncm'] = exportacoes_mundo_2023['Código NCM'].astype(str).str.zfill(8)
exportacoes_usa_2023['cod_ncm'] = exportacoes_usa_2023['Código NCM'].astype(str).str.zfill(8)

In [80]:
exportacoes_mundo_2023.head(2)

,Código NCM,Descrição NCM,2023 - Valor US$ FOB,cod_ncm
0,12019000,"Soja, mesmo triturada, exceto para semeadura",53238560791,12019000
1,27090010,Óleos brutos de petróleo,42611078672,27090010


In [83]:
#Taxa média de câmbio em 2023 - compra
taxa_cambio = 5.39  # Exemplo de taxa média de câmbio em 2023
exportacoes_mundo_2023['R$ FOB Mundo'] = exportacoes_mundo_2023['2023 - Valor US$ FOB'] * taxa_cambio
exportacoes_usa_2023['R$ FOB USA'] = exportacoes_usa_2023['2023 - Valor US$ FOB'] * taxa_cambio

## Agregar por SH6

In [84]:
exportacoes_mundo_2023['SH6'] = exportacoes_mundo_2023['cod_ncm'].str[0:6]
exportacoes_usa_2023['SH6'] = exportacoes_usa_2023['cod_ncm'].str[0:6]

#Agregar os valores por SH6
exportacoes_mundo_2023 = exportacoes_mundo_2023.groupby('SH6')[['R$ FOB Mundo']].sum()
    

In [86]:
exportacoes_usa_2023 = exportacoes_usa_2023.groupby('SH6')[['R$ FOB USA']].sum()    

In [87]:
exportacoes = exportacoes_mundo_2023.merge(exportacoes_usa_2023, on='SH6', how='left').fillna(0)

In [88]:
exportacoes['participacao_usa'] = exportacoes['R$ FOB USA'] / (exportacoes['R$ FOB Mundo'])

In [89]:
exportacoes.head(2)

,R$ FOB Mundo,R$ FOB USA,participacao_usa
SH6,,,
010121,40075081.20,29427610.52,0.734312
010129,21803972.96,15256136.28,0.699695


In [85]:
exportacoes_mundo_2023.head(2)

,R$ FOB Mundo
SH6,
010121,40075081.20
010129,21803972.96


## Mapeamento ISIC e CNAE

In [14]:
ncm_isic = pd.read_csv('./dados/comex/NCM_ISIC (1).csv',sep=';',dtype={'CO_ISIC_CLASSE': str, 'NO_ISIC_CLASSE': str}    )
ncm = pd.read_csv('./dados/comex/NCM (2).csv',sep=';',dtype={'CO_SH6': str, 'CO_ISIC_CLASSE': str})

In [191]:
mapeamento_isic = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])

In [192]:
mapeamento_isic.head(2)

,código ISIC,denominação ISIC,código CNAE,denominação CNAE
0,A,"AGRICULTURA, GANADERÍA, SILVICULTURA Y PESCA",A,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
1,01,"AGRICULTURA, GANADERÍA, CAZA Y ACTIVIDADES DE ...",01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"


In [193]:

#Remmover linhas em que o código ISIC tem comprimento menor que 4
mapeamento_isic = mapeamento_isic[mapeamento_isic['código ISIC'].str.len() == 3]
mapeamento_isic.tail()

,código ISIC,denominação ISIC,código CNAE,denominação CNAE
1171,960,Otras actividades de servicios personales,96.0,Outras atividades de serviços pessoais
1178,970,Actividades de los hogares como empleadores de...,97.0,Serviços domésticos
1181,981,Actividades no diferenciadas de los hogares co...,NaN,NaN
1183,982,Actividades no diferenciadas de los hogares co...,NaN,NaN
1187,990,Actividades de organizaciones y órganos extrat...,99.0,Organismos internacionais e outras instituiçõe...


In [195]:
mapeamento_isic = mapeamento_isic.dropna()

In [190]:
lista_isic_usa_3d  =exportacoes_usa_2023.reset_index().merge(ncm, left_on='SH6', right_on='CO_SH6', how='left')['CO_ISIC_CLASSE'].str[0:3].drop_duplicates()

In [ ]:
#Ver códigos cnae únicos por isic 
mapeamento_isic.groupby('código ISIC')[['código CNAE','denominação ISIC']].drop_duplicates().reset_index().query("`código ISIC`.isin(@lista_isic_usa_3d) ").sort_values('código CNAE', ascending=False)

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [65]:
mapeamento_isic['CNAE2d'] = mapeamento_isic['código CNAE'].str[0:2]
mapeamento_isic['ISIC2d'] = mapeamento_isic['código ISIC'].str[0:2]

In [66]:
mapeamento_isic.groupby('ISIC2d')[['CNAE2d']].nunique().sort_values(by='CNAE2d', ascending=False).head(10)

,CNAE2d
ISIC2d,
16,2
58,2
22,2
70,2
20,2
19,2
88,2
01,1
73,1


In [71]:
mapeamento_isic.query('ISIC2d.isin(["58","70","88"])')

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d
824,5811,Edición de libros,58.11-5,Edição de livros,58,58
825,5811,Edición de libros,58.21-2,Edição integrada à impressão de livros,58,58
826,5812,Edición de directorios y listas de correo,58.19-1,"Edição de cadastros, listas e outros produtos ...",58,58
827,5812,Edición de directorios y listas de correo,58.29-8,"Edição integrada à impressão de cadastros, lis...",58,58
828,5813,"Edición de periódicos, revistas y otras public...",58.12-3,Edição de jornais,58,58
829,5813,"Edición de periódicos, revistas y otras public...",58.13-1,Edição de revistas,58,58
830,5813,"Edición de periódicos, revistas y otras public...",58.22-1,Edição integrada à impressão de jornais,58,58
831,5813,"Edición de periódicos, revistas y otras public...",58.23-9,Edição integrada à impressão de revistas,58,58
832,5819,Otras actividades de edición,58.19-1,"Edição de cadastros, listas e outros produtos ...",58,58
833,5819,Otras actividades de edición,58.29-8,"Edição integrada à impressão de cadastros, lis...",58,58


In [39]:
mapeamento_isic.query('`CNAE2d`.isin(["15"])')

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d
215,1511,Curtido y adobo de cueros; adobo y teñido de p...,15.10-6,Curtimento e outras preparações de couro,15
216,1512,"Fabricación de maletas, bolsos de mano y artíc...",15.21-1,"Fabricação de artigos para viagem, bolsas e se...",15
217,1512,"Fabricación de maletas, bolsos de mano y artíc...",15.29-7,Fabricação de artefatos de couro não especific...,15
220,1520,Fabricación de calzado,15.31-9,Fabricação de calçados de couro,15
221,1520,Fabricación de calzado,15.32-7,Fabricação de tênis de qualquer material,15
222,1520,Fabricación de calzado,15.33-5,Fabricação de calçados de material sintético,15
223,1520,Fabricación de calzado,15.39-4,Fabricação de calçados de materiais não especi...,15
224,1520,Fabricación de calzado,15.40-8,"Fabricação de partes para calçados, de qualque...",15
234,1629,Fabricación de otros productos de madera; fabr...,15.40-8,"Fabricação de partes para calçados, de qualque...",15
319,2219,Fabricación de otros productos de caucho,15.40-8,"Fabricação de partes para calçados, de qualque...",15


In [58]:
ncms_usa = exportacoes_usa_2023[['Código NCM']].drop_duplicates().merge(ncm, left_on='Código NCM', right_on='CO_NCM', how='left').drop('Código NCM', axis=1)

Há produtos exportados para os EUA que não estão mapeados no NCM_ISIC, como por exemplo:

In [64]:
mapeamento_isic.query('`código ISIC`.isin(["2220","2219","1629","2011"])')

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d
234,1629,Fabricación de otros productos de madera; fabr...,15.40-8,"Fabricação de partes para calçados, de qualque...",15
235,1629,Fabricación de otros productos de madera; fabr...,16.29-3,"Fabricação de artefatos de madeira, palha, cor...",16
274,2011,Fabricación de sustancias químicas básicas,19.31-4,Fabricação de álcool,19
275,2011,Fabricación de sustancias químicas básicas,19.32-2,"Fabricação de biocombustíveis, exceto álcool",19
276,2011,Fabricación de sustancias químicas básicas,20.11-8,Fabricação de cloro e álcalis,20
277,2011,Fabricación de sustancias químicas básicas,20.14-2,Fabricação de gases industriais,20
278,2011,Fabricación de sustancias químicas básicas,20.19-3,Fabricação de produtos químicos inorgânicos nã...,20
279,2011,Fabricación de sustancias químicas básicas,20.21-5,Fabricação de produtos petroquímicos básicos,20
280,2011,Fabricación de sustancias químicas básicas,20.22-3,Fabricação de intermediários para plastificant...,20
281,2011,Fabricación de sustancias químicas básicas,20.29-1,Fabricação de produtos químicos orgânicos não ...,20


In [ ]:
#Fazer o count distinct de códigos CNAE2d por código ISIC
mapeamento_isic.merge(ncms_usa[['CO_ISIC_CLASSE']].rename(columns={'CO_ISIC_CLASSE': 'código ISIC'}).drop_duplicates(),on='código ISIC',how='right',indicator=True).groupby('ISIC2d')['CNAE2d'].nunique().reset_index().sort_values(by='CNAE2d', ascending=False).head(10)


,ISIC2d,CNAE2d
18,22,2
16,20,2
12,16,2
25,29,1
20,24,1
21,25,1
22,26,1
23,27,1
24,28,1
0,01,1


## Decidiu-se por usar os dois primeiros dígitos do ISIC como aproximação da CNAE

In [101]:
exportacoes = exportacoes.query('`R$ FOB USA` > 0')

In [114]:
exportacoes = ncm[['CO_SH6','CO_ISIC_CLASSE']].drop_duplicates().merge(exportacoes, left_on='CO_SH6', right_on='SH6', how='inner')

In [115]:
exportacoes['CNAE2s'] = exportacoes['CO_ISIC_CLASSE'].str[0:2]

In [116]:
exportacoes.head(2)

,CO_SH6,CO_ISIC_CLASSE,R$ FOB Mundo,R$ FOB USA,participacao_usa,CNAE2s
0,490110,5811,6996257.73,701988.21,0.100338,58
1,490191,5811,2892376.41,73713.64,0.025485,58


## Carregar dados da PIA Empresa

In [196]:
url_pia_empresa = 'https://apisidra.ibge.gov.br/values/t/1842/n1/all/v/824,825/p/last%201/c12762/all'

data = requests.get(url_pia_empresa)
data = data.json()
pia_empresa_rlv = pd.DataFrame(data, columns=data[0])


In [197]:
pia_empresa_rlv.columns = pia_empresa_rlv.iloc[0]
pia_empresa_rlv = pia_empresa_rlv[1:]


In [198]:
pia_empresa_rlv.head(2)


,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Ano (Código),Ano,Classificação Nacional de Atividades Econômicas (CNAE 2.0) (Código),Classificação Nacional de Atividades Econômicas (CNAE 2.0)
1,1,Brasil,40,Mil Reais,6353568639,1,Brasil,824,Total da receita líquida de vendas,2023,2023,117897,Total
2,1,Brasil,40,Mil Reais,456258347,1,Brasil,824,Total da receita líquida de vendas,2023,2023,116880,B Indústrias extrativas


In [ ]:
mapeamento_isic

In [199]:
##Filtrar apenas registros cujos valores da coluna 'Classificação Nacional de Atividades Econômicas (CNAE 2.0)' siga o padrão \d{2}\.\d{1} 
pia_empresa_rlv_2d = pia_empresa_rlv[pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str.match(r'^\d{2} ')]
pia_empresa_rlv_3d = pia_empresa_rlv[pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str.match(r'^\d{2}\.\d{1} ')]    

In [201]:
pia_empresa_rlv_3d.head(2)

,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Ano (Código),Ano,Classificação Nacional de Atividades Econômicas (CNAE 2.0) (Código),Classificação Nacional de Atividades Econômicas (CNAE 2.0)
4,1,Brasil,40,Mil Reais,1941673,1,Brasil,824,Total da receita líquida de vendas,2023,2023,116882,05.0 Extração de carvão mineral
6,1,Brasil,40,Mil Reais,163281926,1,Brasil,824,Total da receita líquida de vendas,2023,2023,116885,06.0 Extração de petróleo e gás natural


In [202]:
pia_empresa_rlv_2d['cnae'] = pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[0:2].str.replace('.','', regex=False)
pia_empresa_rlv_2d['cnae_descricao'] = pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[3:]

pia_empresa_rlv_2d.head(2)



/tmp/ipykernel_1101985/1836566463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pia_empresa_rlv_2d['cnae'] = pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[0:2].str.replace('.','', regex=False)
/tmp/ipykernel_1101985/1836566463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pia_empresa_rlv_2d['cnae_descricao'] = pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[3:]


,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Ano (Código),Ano,Classificação Nacional de Atividades Econômicas (CNAE 2.0) (Código),Classificação Nacional de Atividades Econômicas (CNAE 2.0),cnae,cnae_descricao
3,1,Brasil,40,Mil Reais,1941673,1,Brasil,824,Total da receita líquida de vendas,2023,2023,116881,05 Extração de carvão mineral,05,Extração de carvão mineral
5,1,Brasil,40,Mil Reais,163281926,1,Brasil,824,Total da receita líquida de vendas,2023,2023,116884,06 Extração de petróleo e gás natural,06,Extração de petróleo e gás natural


In [203]:
pia_empresa_rlv_3d['cnae'] = pia_empresa_rlv_3d['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[0:4].str.replace('.','', regex=False)
pia_empresa_rlv_3d['cnae_descricao'] = pia_empresa_rlv_3d['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[5:]

/tmp/ipykernel_1101985/1737546974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pia_empresa_rlv_3d['cnae'] = pia_empresa_rlv_3d['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[0:4].str.replace('.','', regex=False)
/tmp/ipykernel_1101985/1737546974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pia_empresa_rlv_3d['cnae_descricao'] = pia_empresa_rlv_3d['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[5:]


In [233]:
pia_empresa_rlv = pd.concat([pia_empresa_rlv_2d[['Variável','cnae','cnae_descricao','Valor']], pia_empresa_rlv_3d[['Variável','cnae','cnae_descricao','Valor']]], ignore_index=True)

In [235]:
pia_empresa_rlv.tail()

,Variável,cnae,cnae_descricao,Valor
275,Receita líquida de vendas de produtos e serviç...,324,Fabricação de brinquedos e jogos recreativos,3297487
276,Receita líquida de vendas de produtos e serviç...,325,Fabricação de instrumentos e materiais para us...,20777571
277,Receita líquida de vendas de produtos e serviç...,329,Fabricação de produtos diversos,15396002
278,Receita líquida de vendas de produtos e serviç...,331,Manutenção e reparação de máquinas e equipamentos,50921841
279,Receita líquida de vendas de produtos e serviç...,332,Instalação de máquinas e equipamentos,6735735


In [237]:
#Linhas em que o valor da coluna 'Valor' não pode ser convertido para float, trocar por -1
pia_empresa_rlv['Valor'] = pd.to_numeric(pia_empresa_rlv['Valor'], errors='coerce').fillna(-1)
pia_empresa_rlv['Valor'] = pia_empresa_rlv['Valor']*1000
#Fazer o pivot transformando a coluna 'Variável' em colunas e preenchendo com os valores da coluna 'Valor'
pia_empresa_rlv = pia_empresa_rlv.pivot_table(index=['cnae','cnae_descricao'], columns='Variável', values='Valor')
pia_empresa_rlv.reset_index(inplace=True)

In [216]:
mapeamento_isic['cnae'] = mapeamento_isic['código CNAE'].str.replace('.','', regex=False)

In [221]:
exportacoes_pia = exportacoes.copy()

In [223]:
lista_codigos_isic_mapeados_multiplas_cnae_3d = pia_empresa_rlv_3d[['cnae']].drop_duplicates().merge(mapeamento_isic, on='cnae', how='left').groupby('código ISIC')[['código CNAE']].nunique().reset_index().query("`código CNAE` > 1")['código ISIC'].unique().tolist()

In [ ]:
#criar uma coluna 'cnae' em exportacoes_pia e fazer com que seu valor  seja igual aos 2 dígitos da coluna CO_ISIC_CLASSE, caso os 3 primeiros dígitos do código isic classe fizerem parte de lista_codigos_isic_mapeados_multiplas_cnae_3d
# caso contrário, usar None
exportacoes_pia['cnae'] = exportacoes_pia['CO_ISIC_CLASSE'].str[0:2].where(
    exportacoes_pia['CO_ISIC_CLASSE'].str[0:3].isin(lista_codigos_isic_mapeados_multiplas_cnae_3d),
    None
)

In [229]:
mapeamento_isic.head()

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,cnae
2,011,Cultivo de plantas no perenes,01.1,Produção de lavouras temporárias,011
3,011,Cultivo de plantas no perenes,01.2,Horticultura e floricultura,012
4,011,Cultivo de plantas no perenes,01.3,Produção de lavouras permanentes,013
5,011,Cultivo de plantas no perenes,01.4,Produção de sementes e mudas certificadas,014
25,012,Cultivo de plantas perennes,01.1,Produção de lavouras temporárias,011


In [ ]:
mapeamento_isic

In [231]:
#Nos campos None em cnae, usar o valor da coluna CO_ISIC_CLASSE e mapear os 3 primeiros dígitos para o código CNAE usando como mapeamento o mapeamento_isic
mapeamento_isic_unico = mapeamento_isic.drop_duplicates(subset='código ISIC')

exportacoes_pia['cnae'] = exportacoes_pia['cnae'].fillna(
    exportacoes_pia['CO_ISIC_CLASSE'].str[0:3].map(
        mapeamento_isic_unico.set_index('código ISIC')['cnae']
    )
)

In [232]:
exportacoes_pia.head()

,CO_SH6,CO_ISIC_CLASSE,R$ FOB Mundo,R$ FOB USA,participacao_usa,CNAE2s,cnae
0,490110,5811,6.996258e+06,701988.21,0.100338,58,581
1,490191,5811,2.892376e+06,73713.64,0.025485,58,581
2,490199,5811,1.819830e+08,19922588.07,0.109475,58,581
3,521029,1312,4.218591e+05,9055.20,0.021465,13,13
4,521032,1312,7.247394e+05,7702.31,0.010628,13,13


In [244]:
exportacoes_pia.head()

,CO_SH6,CO_ISIC_CLASSE,R$ FOB Mundo,R$ FOB USA,participacao_usa,CNAE2s,cnae
0,490110,5811,6.996258e+06,701988.21,0.100338,58,581
1,490191,5811,2.892376e+06,73713.64,0.025485,58,581
2,490199,5811,1.819830e+08,19922588.07,0.109475,58,581
3,521029,1312,4.218591e+05,9055.20,0.021465,13,13
4,521032,1312,7.247394e+05,7702.31,0.010628,13,13


In [273]:
exportacoes_pia = exportacoes_pia.groupby('cnae')[['R$ FOB Mundo','R$ FOB USA']].sum().reset_index().merge(pia_empresa_rlv, on='cnae', how='inner')

In [ ]:
exportacoes_pia['Participação USA'] = exportacoes_pia['R$ FOB USA'] / (exportacoes_pia['R$ FOB Mundo'] + exportacoes_pia['R$ FOB USA'])

,cnae,R$ FOB Mundo,R$ FOB USA,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas
0,060,2.299204e+11,2.525145e+10,Extração de petróleo e gás natural,1.626363e+14,1.632819e+14
1,071,1.648986e+11,2.071770e+09,Extração de minério de ferro,1.928306e+14,1.969583e+14


In [268]:
exportacoes_pia.query('CO_ISIC_CLASSE.str.startswith("10")')

,CO_SH6,CO_ISIC_CLASSE,R$ FOB Mundo,R$ FOB USA,participacao_usa,CNAE2s,cnae
147,020714,1010,3.677516e+10,2205038.22,0.000060,10,101
182,030520,1020,4.744817e+04,38419.92,0.809724,10,102
183,190120,1061,2.735497e+08,39761097.53,0.145352,10,106
184,071290,1030,2.464648e+06,63192.36,0.025640,10,103
205,350510,1062,3.037367e+08,21755894.16,0.071627,10,106
...,...,...,...,...,...,...,...
2348,200941,1030,8.977584e+04,6726.72,0.074928,10,103
2349,200949,1030,7.185743e+07,6936951.56,0.096538,10,103
2350,200950,1030,5.971581e+05,140.14,0.000235,10,103
2351,200961,1030,1.311906e+07,2844313.78,0.216808,10,103


In [277]:
exportacao_pia = pia_empresa_rlv.merge(exportacoes, left_on='cnae', right_on='CNAE2s', how='inner')

In [286]:
#Criar uma coluna de 'Intensidade Tecnloógica' de acordo com a classificação de intensidade tecnológica da OCDE, mapeada por ISIC, onde a os primeiros dígitos ISIC (2 ou 3 ) podem ser mapeados de acordo com a tabela ISIC x Intensidade Tecnológica é:
# Alta Intensidade Tecnológica: 303, 21, 26,72, 582
# Média Alta Intensidade Tecnológica: 252, 325, 28, 20, 27, 302, 304, 309,62-63
# Média Intensidade Tecnológica: 22, 301, 32 ( com exceção de 325),23, 24, 33
# Média Baixa Intensidade Tecnológica: 10-12,13, 14, 15, 25 ( menos 252),16, 18 19, 31
# Baixa Intensidade Tecnológica: Demais
#Considere que o código ISIC tabela de mapeamento pode ter 2 ou 3 dígitos, que devem corresponder aos primeiros 2 ou 3 dígitos do código ISIC da coluna CO_ISIC_CLASSE, ou seja,
#mapeie inicialmente os códigos com 3 dígitos e em seguida os códigos com 2 dígitos, caso não tenha sido mapeado ainda.
mapeamento_3d = {
    '303': 'Alta Intensidade Tecnológica',
    '582': 'Alta Intensidade Tecnológica',

    '302': 'Média Alta Intensidade Tecnológica',
    '252': 'Média Alta Intensidade Tecnológica',
    '304': 'Média Alta Intensidade Tecnológica',
    '325': 'Média Alta Intensidade Tecnológica',
    '309': 'Média Alta Intensidade Tecnológica',

    '301': 'Média Intensidade Tecnológica',
    
    '581':'Média Baixa Intensidade Tecnológica'
}


# Mapear códigos de 2 dígitos
mapeamento_2d = {
    '21': 'Alta Intensidade Tecnológica',
    '26': 'Alta Intensidade Tecnológica',
    '72': 'Alta Intensidade Tecnológica',

    '29': 'Média Alta Intensidade Tecnológica',
    '28': 'Média Alta Intensidade Tecnológica',
    '20': 'Média Alta Intensidade Tecnológica',
    '27': 'Média Alta Intensidade Tecnológica',
    '62': 'Média Alta Intensidade Tecnológica',
    '63': 'Média Alta Intensidade Tecnológica',

    '22': 'Média Intensidade Tecnológica',
    '32': 'Média Intensidade Tecnológica',
    '23': 'Média Intensidade Tecnológica',
    '24': 'Média Intensidade Tecnológica',
    '25': 'Média Intensidade Tecnológica',
    '33': 'Média Intensidade Tecnológica',

    '10': 'Média Baixa Intensidade Tecnológica',
    '11': 'Média Baixa Intensidade Tecnológica',
    '12': 'Média Baixa Intensidade Tecnológica',
    '13': 'Média Baixa Intensidade Tecnológica',
    '14': 'Média Baixa Intensidade Tecnológica',
    '15': 'Média Baixa Intensidade Tecnológica',
    '16': 'Média Baixa Intensidade Tecnológica',
    '17': 'Média Baixa Intensidade Tecnológica',
    '18': 'Média Baixa Intensidade Tecnológica',
    '19': 'Média Baixa Intensidade Tecnológica',
    '31': 'Média Baixa Intensidade Tecnológica',
    #69 a 75 Média Baixa Intensidade Tecnológica
    '69': 'Média Baixa Intensidade Tecnológica',
    '70': 'Média Baixa Intensidade Tecnológica',
    '71': 'Média Baixa Intensidade Tecnológica',
    '73': 'Média Baixa Intensidade Tecnológica',
    '74': 'Média Baixa Intensidade Tecnológica',
    '75': 'Média Baixa Intensidade Tecnológica',
    '61' : 'Média Baixa Intensidade Tecnológica',
    #05 a 09 Média Baixa Intensidade Tecnológica
    '05': 'Média Baixa Intensidade Tecnológica',
    '06': 'Média Baixa Intensidade Tecnológica',
    '07': 'Média Baixa Intensidade Tecnológica',
    '08': 'Média Baixa Intensidade Tecnológica',
    '09': 'Média Baixa Intensidade Tecnológica'

}



In [ ]:
exportacao_pia.drop(columns='Intensidade Tecnológica', inplace=True, errors='ignore')

In [287]:
# Aplicar mapeamento primeiro por 3 dígitos, depois por 2 dígitos
exportacao_pia['Intensidade Tecnológica'] = exportacao_pia['CO_ISIC_CLASSE'].str[0:3].map(mapeamento_3d)

# Para valores não mapeados (NaN), tentar mapear por 2 dígitos
mask_nao_mapeado = exportacao_pia['Intensidade Tecnológica'].isna()
exportacao_pia.loc[mask_nao_mapeado, 'Intensidade Tecnológica'] = exportacao_pia.loc[mask_nao_mapeado, 'CO_ISIC_CLASSE'].str[0:2].map(mapeamento_2d)

# Exceção: 252 deve ser removido de '25' (que já foi mapeado como Média Alta)
#mask_252 = exportacao_pia['CO_ISIC_CLASSE'].str.startswith('252')
#exportacao_pia.loc[mask_252, 'Intensidade Tecnológica'] = 'Baixa Intensidade Tecnológica'

# Para códigos não mapeados, classificar como 'Baixa Intensidade Tecnológica'
exportacao_pia['Intensidade Tecnológica'] = exportacao_pia['Intensidade Tecnológica'].fillna('Baixa Intensidade Tecnológica')

In [304]:
exportacao_pia.query('CO_ISIC_CLASSE.str.startswith("63")').head()

,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,CO_SH6,CO_ISIC_CLASSE,R$ FOB Mundo,R$ FOB USA,participacao_usa,CNAE2s,Intensidade Tecnológica


In [306]:
exportacoes.shape

(3455, 6)

In [305]:
exportacao_pia.shape

(3222, 11)

In [307]:
exportacao_pia.to_excel('./Outputs/final/exportacao_pia_intensidade.xlsx', index=False)